In [24]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
import pycrfsuite
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
import random
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix, plot_confusion_matrix, ConfusionMatrixDisplay
import os
import matplotlib.pyplot as plt

In [39]:
nlp = English()
    # Create a blank Tokenizer with just the English vocab
tokenizer = Tokenizer(nlp.vocab)
# filepath = '../../data/conll2003/eng.all'
filepath = '../../data/SEC-filings/CONLL-format/data/eng.all'
sec_data = read_text_labeled_sentences(filepath, tokenizer)
random.shuffle(sec_data)
main(sec_data)



model's parameters : ['feature.minfreq', 'feature.possible_states', 'feature.possible_transitions', 'c1', 'c2', 'max_iterations', 'num_memories', 'epsilon', 'period', 'delta', 'linesearch', 'max_linesearch']
Last iteration log {'num': 60, 'scores': {}, 'loss': 532.023099, 'feature_norm': 40.440112, 'error_norm': 93.032465, 'active_features': 1575, 'linesearch_trials': 1, 'linesearch_step': 1.0, 'time': 0.037}
sentence: NO PLEDGE OR LOAN ; CHARACTERIZATION . 
predicted labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O']
real labels ['O', 'O', 'O', 'O', 'O', 'O', 'O']


ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead - the MultiLabelBinarizer transformer can convert to this format.

In [40]:
def read_text_labeled_sentences(filepath, tokenizer):
    with open(filepath) as f:
        lines = f.readlines() # list containing lines of file
    sentence = ""
    labels = []
    TRAINING_DATA = []
    for line in lines:
        if line.startswith('-DOCSTART-'): continue
        elif line == '\n':
            if labels == [] and sentence == "": continue
            tokens = tokenizer(sentence.lstrip())
            TRAINING_DATA.append((tokens,labels))
            sentence = ""
            labels = []
            
        else: 
            elements = line.split(' ')
#             elements = line.split('\t')
            word = elements[0]
            label = elements[3][:-1]
#             label = elements[1]
            labels.append(label)
            sentence += word + ' '
    return TRAINING_DATA

def main(data):
    # Divide disease dataset into train and test sets
    training_data = data[int(0.3*len(data)):]
    test_data = data[:int(0.3*len(data))]
    # Calculate features for both training and test datasets
    X_train = [sent2features(s[0]) for s in training_data]
    y_train = [s[1] for s in training_data]
    X_test = [sent2features(s[0]) for s in test_data]
    y_test = [s[1] for s in test_data]
    # Train the model
    train(X_train, y_train)

    # Predict labels for a given sentence example
    tagger = pycrfsuite.Tagger()
    tagger.open("../CRF_NER/models/anonmodel.crfsuite")
    i=0
    print("sentence: {}".format(test_data[i][0]))
    print("predicted labels: {}". format(tagger.tag(X_test[i])))
    print("real labels {}".format(y_test[i]))
    
    # Calculate test metrics
    outputs = []
    for i in range(len(X_test)):
        outputs.append(tagger.tag(X_test[i]))

    targets = sum(y_test, [])
    outputs = sum(outputs, [])
    
#     print("conf_matrix: \n", confusion_matrix(targets, outputs))
    print("accuracy score:\n", accuracy_score(targets, outputs))
    print("precision score:\n", precision_score(targets, outputs, average='micro'))
    print("recall score:\n", recall_score(targets, outputs, average='micro'))
    print("F1 score:\n", f1_score(targets, outputs, average='micro'))

def word2features(train_sample, i):
    token = train_sample[i]
    word = token.text
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'word.pos='+token.pos_,
        'word.dep='+token.dep_,
        'word.is_stop=%s' %token.is_stop,
        'word.lemma=' + token.lemma_,
        'word.tag=' + token.tag_,
        'word.shape=' + token.shape_,
        'word.is_alpha=%s' %token.is_alpha,        
    ]
    if i > 0:
        token1 = train_sample[i-1]
        word1 = token1.text
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.pos='+token1.pos_,
            '-1:word.dep='+token1.dep_,
            '-1:word.is_stop=%s' %token1.is_stop,
            '-1:word.lemma=' + token1.lemma_,
            '-1:word.tag=' + token1.tag_,
            '-1:word.shape=' + token1.shape_,
            '-1:word.is_alpha=%s' %token1.is_alpha,    
        ])
    else:
        features.append('BOS')
        
    if i < len(train_sample)-1:
        token1 = train_sample[i+1]
        word1 = token1.text
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.pos='+token1.pos_,
            '+1:word.dep='+token1.dep_,
            '+1:word.is_stop=%s' %token1.is_stop,
            '+1:word.lemma=' + token1.lemma_,
            '+1:word.tag=' + token1.tag_,
            '+1:word.shape=' + token1.shape_,
            '+1:word.is_alpha=%s' %token1.is_alpha,   
        ])
    else:
        features.append('EOS')       
    return features

def sent2features(train_sample):
    return [word2features(train_sample, i) for i in range(len(train_sample))]

def train(X_train, y_train, model_path="../CRF_NER/models/anonmodel.crfsuite"):
    trainer = pycrfsuite.Trainer(verbose=False)

    for xseq, yseq in zip(X_train, y_train):
        trainer.append(xseq, yseq)

    trainer.set_params({
        'c1': 0.44,   # coefficient for L1 penalty
        'c2': 1e-4,  # coefficient for L2 penalty
        'max_iterations': 60,  # stop earlier

        # include transitions that are possible, but not observed
        'feature.possible_transitions': True
    })
    print("model's parameters : {}".format(trainer.params()))
    trainer.train(model_path)
    print("Last iteration log {}".format(trainer.logparser.last_iteration))